# Gioco del tris

In [97]:
import numpy as np # Per -inf e +inf
import time # Per il tempo stimato

In [98]:
class Game:

    # COSTRUTTORE
    def __init__(self):
        # Inizializza il tabellone
        self.current = [['.','.','.'],
                        ['.','.','.'],
                        ['.','.','.']]
        self.player_turn = 'X' # Si inzia sempre con X

    # STAMPA IL TABELLONE
    def print_board(self):
        for ind in range(0, 3):
            for inx in range(0, 3):
                print('{}|'.format(self.current[ind][inx]), end=" ")
            print() # Fine riga: vai a capo
        print() # Vai a capo non mischiare i tabelloni

    # VERIFICA LA VALIDITA' DELLA MOSSA
    def is_valid(self, ind, inx):
        if ((ind < 0 or ind > 2 or inx < 0 or inx > 2) or # fuori
            self.current[ind][inx] != '.'): # casella occupata
            return False
        return True # Mossa valida
    
    # GIOCO FINITO
    def is_end(self):
        # Vittoria su una colonna
        # |X | |
        # |X | |
        # |X | |
        for ind in range(0,3):
            if (self.current[0][ind] != '.' and # non e' vuota
                # Stesso segno nella stessa colonna
                self.current[0][ind] == self.current[1][ind] and
                self.current[1][ind] == self.current[2][ind]):
                return self.current[0][ind] # ritorna il segno
        # Vittoria su una riga
        # |X |X |X
        # |  |  |
        # |  |  |
        for ind in range(0, 3):
            if self.current[ind] == ['X', 'X', 'X']:
                return 'X' # X ha vinto
            if self.current[ind] == ['O', 'O', 'O']:
                return 'O' # O ha vinto
        # Vittoria sulla diagonale principale
        # |X |  | 
        # |  |X |
        # |  |  |X
        if (self.current[0][0] != '.' and
            self.current[0][0] == self.current[1][1] and
            self.current[1][1] == self.current[2][2]):
            return self.current[0][0]
        # Vittoria sulla diagonale secondaria
        # |  |  |X 
        # |  |X |
        # |X |  |
        if (self.current[0][2] != '.' and
            self.current[0][2] == self.current[1][1] and
            self.current[1][1] == self.current[2][0]):
            return self.current[0][2]
        # Verifica se il tabellone non sia pieno
        # |X |O |X 
        # |O |O |X
        # |X |X |
        for ind in range(0, 3):
            for inx in range(0, 3):
                if self.current[ind][inx] == '.':
                    # Il gioco non e' finito
                    return None
        # Partita patta
        # |X |O |X 
        # |O |O |X
        # |X |X |O
        return 'patta'
    
    # -------
    # MIN-MAX
    # -------
    def max(self):
        # Giocatore O
        maxv = -np.inf # all'inizio ponilo a -inf
        # Posizioni
        px = None
        py = None
        result = self.is_end()
        if result == 'X':
            return (-1, 0, 0) # Sconfitta
        elif result == 'O':
            return (1, 0, 0) # Vittoria
        elif result == 'patta':
            return (0, 0, 0) # Patta
        
        # Esecuzione dell'algoritmo
        for ind in range(0, 3):
            for inx in range(0, 3):
                if self.current[ind][inx] == '.':
                    # Trovata casalla vuota:
                    # inserisci O
                    # esegui il min
                    self.current[ind][inx] = 'O'
                    (m, min_x, min_y) = self.min()
                    if m > maxv:
                        # Trovato un valore migliore
                        maxv = m
                        px = ind
                        py = inx
                    # Imposta di nuovo la casella vuota
                    self.current[ind][inx] = '.'
        return (maxv, px, py)

    def min(self):
        # Giocatore X
        minv = np.inf # all'inizio ponilo a +inf
        # Posizioni
        qx = None
        qy = None
        result = self.is_end()
        if result == 'X':
            return (1, 0, 0) # Vittoria
        elif result == 'O':
            return (-1, 0, 0) # Sconfitta
        elif result == 'patta':
            return (0, 0, 0) # Patta
        
        # Esecuzione dell'algoritmo
        for ind in range(0, 3):
            for inx in range(0, 3):
                if self.current[ind][inx] == '.':
                    # Trovata casalla vuota:
                    # inserisci O
                    # esegui il min
                    self.current[ind][inx] = 'X'
                    (m, max_x, max_y) = self.max()
                    if m < minv:
                        # Trovato un valore migliore
                        minv = m
                        qx = ind
                        qy = inx
                    # Imposta di nuovo la casella vuota
                    self.current[ind][inx] = '.'
        return (minv, qx, qy)

    def play_min_max(self):
        while True:
            self.print_board()
            self.result = self.is_end()

            # Stampa dell'appropriato messaggio se il gioco è finito
            if self.result != None:
                if self.result == 'X':
                    print('Il vincitore è X!')
                elif self.result == 'O':
                    print('Il vincitore è O!')
                elif self.result == 'patta':
                    print('La partita è patta!')
                return

            # E' il turno X
            if self.player_turn == 'X':
                while True:

                    start = time.time()
                    (m, qx, qy) = self.min()
                    end = time.time()
                    print('Evaluation time: {}s'.format(round(end - start, 7)))
                    print('Mossa raccomandata: X = {}, Y = {}'.format(qx, qy))

                    px = int(input('Inserisci l\'indice di riga: '))
                    py = int(input('Inserisci l\'indice di colonna: '))

                    if self.is_valid(px, py):
                        self.current[px][py] = 'X'
                        self.player_turn = 'O'
                        break
                    else:
                        print('La mossa non è valida! Prova ancora.')

            # E' il turno di O
            else:
                (m, px, py) = self.max()
                self.current[px][py] = 'O'
                self.player_turn = 'X'

    # -----------------------------
    # MIN-MAX CON ALFA-BETA PRUNING
    # -----------------------------
    def max_alfa_beta(self, alfa, beta):
        # Giocatore O
        maxv = -np.inf # all'inizio ponilo a -inf
        # Posizioni
        px = None
        py = None
        result = self.is_end()
        if result == 'X':
            return (-1, 0, 0) # Sconfitta
        elif result == 'O':
            return (1, 0, 0) # Vittoria
        elif result == 'patta':
            return (0, 0, 0) # Patta
        
        # Esecuzione dell'algoritmo
        for ind in range(0, 3):
            for inx in range(0, 3):
                if self.current[ind][inx] == '.':
                    # Trovata casalla vuota:
                    # inserisci O
                    # esegui il min
                    self.current[ind][inx] = 'O'
                    (m, min_x, min_y) = self.min_alfa_beta(alfa, beta)
                    if m > maxv:
                        # Trovato un valore migliore
                        maxv = m
                        px = ind
                        py = inx
                    # Imposta di nuovo la casella vuota
                    self.current[ind][inx] = '.'
                    if maxv >= beta:
                        # maxv e' un maggiorante: potatura
                        return (maxv, px, py)
                    if maxv > alfa:
                        # Aggiorna alfa
                        alfa = maxv
        return (maxv, px, py)

    def min_alfa_beta(self, alfa, beta):
        # Giocatore X
        minv = np.inf # all'inizio ponilo a +inf
        # Posizioni
        qx = None
        qy = None
        result = self.is_end()
        if result == 'X':
            return (1, 0, 0) # Vittoria
        elif result == 'O':
            return (-1, 0, 0) # Sconfitta
        elif result == 'patta':
            return (0, 0, 0) # Patta
        
        # Esecuzione dell'algoritmo
        for ind in range(0, 3):
            for inx in range(0, 3):
                if self.current[ind][inx] == '.':
                    # Trovata casalla vuota:
                    # inserisci O
                    # esegui il min
                    self.current[ind][inx] = 'X'
                    (m, max_x, max_y) = self.max_alfa_beta(alfa, beta)
                    if m < minv:
                        # Trovato un valore migliore
                        minv = m
                        qx = ind
                        qy = inx
                    # Imposta di nuovo la casella vuota
                    self.current[ind][inx] = '.'
                    if minv <= alfa:
                        # minv e' un minorante: potatura
                        return (minv, qx, qy)
                    if minv < beta:
                        # Aggiorna beta
                        beta = minv
        return (minv, qx, qy)

    def play_alpha_beta(self):
        while True:
            self.print_board()
            self.result = self.is_end()

            if self.result != None:
                if self.result == 'X':
                    print('Il vincitore è X!')
                elif self.result == 'O':
                    print('Il vincitore è O!')
                elif self.result == 'patta':
                    print('La partita è patta!')
                return

            if self.player_turn == 'X':

                while True:
                    start = time.time()
                    (m, qx, qy) = self.min_alfa_beta(-2, 2)
                    end = time.time()
                    print('Evaluation time: {}s'.format(round(end - start, 7)))
                    print('Mossa raccomandata: X = {}, Y = {}'.format(qx, qy))

                    px = int(input('Inserisci l\'indice di riga: '))
                    py = int(input('Inserisci l\'indice di colonna: '))

                    if self.is_valid(px, py):
                        self.current[px][py] = 'X'
                        self.player_turn = 'O'
                        break
                    else:
                        print('La mossa non è valida! Prova ancora.')

            else:
                (m, px, py) = self.max_alfa_beta(-2, 2)
                self.current[px][py] = 'O'
                self.player_turn = 'X'    

## Main mini-max

In [99]:
g = Game()
g.play_min_max()

.| .| .| 
.| .| .| 
.| .| .| 

Evaluation time: 0.6388731s
Mossa raccomandata: X = 0, Y = 0
X| .| .| 
.| .| .| 
.| .| .| 

X| O| .| 
.| .| .| 
.| .| .| 

Evaluation time: 0.0094821s
Mossa raccomandata: X = 0, Y = 2
X| O| X| 
.| .| .| 
.| .| .| 

X| O| X| 
O| .| .| 
.| .| .| 

Evaluation time: 0.0007191s
Mossa raccomandata: X = 1, Y = 1
X| O| X| 
O| X| .| 
.| .| .| 

X| O| X| 
O| X| O| 
.| .| .| 

Evaluation time: 3.29e-05s
Mossa raccomandata: X = 2, Y = 0
La mossa non è valida! Prova ancora.
Evaluation time: 5.82e-05s
Mossa raccomandata: X = 2, Y = 0
X| O| X| 
O| X| O| 
X| .| .| 

Il vincitore è X!


## Main mini-max con alfa-beta pruning

In [100]:
g = Game()
g.play_alpha_beta()

.| .| .| 
.| .| .| 
.| .| .| 

Evaluation time: 0.0379941s
Mossa raccomandata: X = 0, Y = 0
X| .| .| 
.| .| .| 
.| .| .| 

X| O| .| 
.| .| .| 
.| .| .| 

Evaluation time: 0.0033352s
Mossa raccomandata: X = 0, Y = 2
X| O| X| 
.| .| .| 
.| .| .| 

X| O| X| 
O| .| .| 
.| .| .| 

Evaluation time: 0.0002382s
Mossa raccomandata: X = 1, Y = 1
X| O| X| 
O| X| .| 
.| .| .| 

X| O| X| 
O| X| O| 
.| .| .| 

Evaluation time: 2.31e-05s
Mossa raccomandata: X = 2, Y = 0
X| O| X| 
O| X| O| 
X| .| .| 

Il vincitore è X!
